In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [14]:
import pickle
import tiktoken
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.display import display, Markdown
from langchain.docstore.document import Document
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.retrievers import TFIDFRetriever
from langchain.chat_models import ChatOpenAI

In [15]:
def evaluate_dataframe_answer(df):
    # Add right_or_wrong column if it doesn't exist
    if 'right_or_wrong' not in df.columns:
        df['right_or_wrong'] = ''

    # Widgets
    button_right = widgets.Button(description="Right")
    button_wrong = widgets.Button(description="Wrong")
    button_partial = widgets.Button(description="partial")
    out = widgets.Output()

    # Starting index
    index = [0]  # Use list to allow mutation inside functions

    def display_row(row):
        """
        Display the row data in markdown
        """
        with out:
            display(Markdown(f"**Question**: {row['question']}"))
            display(Markdown(f"**Answer**: {row['answer']}"))
            display(Markdown(f"**Result**: {row['result']}\n"))

    def on_button_right_clicked(b):
        """
        Update right_or_wrong column with 'right' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'right'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_wrong_clicked(b):
        """
        Update right_or_wrong column with 'wrong' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'wrong'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    def on_button_partial_clicked(b):
        """
        Update right_or_wrong column with 'partial' and go to next row
        """
        df.at[index[0], 'right_or_wrong'] = 'partial'
        index[0] += 1
        out.clear_output()
        if index[0] < len(df):
            display_row(df.iloc[index[0]])

    button_right.on_click(on_button_right_clicked)
    button_wrong.on_click(on_button_wrong_clicked)
    button_partial.on_click(on_button_partial_clicked)

    display(out)
    display(button_right, button_wrong, button_partial)

    # Start by displaying the first row
    display_row(df.iloc[0])

In [3]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [4]:
class CustomeSplitter:
    def __init__(self, chunk_threshold=8000, chunk_size=8000, chunk_overlap=50):
        self.chunk_threshold = chunk_threshold
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.enc = tiktoken.get_encoding("cl100k_base")
        self.splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    def token_counter(self, document):
        tokens = self.enc.encode(document.page_content)
        return len(tokens)

    def split(self, documents):
        chunked_documents = []
        for i, doc in enumerate(documents):
            try:
                if self.token_counter(doc) > self.chunk_threshold:
                    chunks = self.splitter.split_documents(doc)
                    chunked_documents.extend(chunks)
                else:
                    chunked_documents.append(doc)
            except:
                chunked_documents.append(doc)
                print(f"Error on document {i}")
                print(doc)

        return chunked_documents

In [5]:
splitter = CustomeSplitter()
chunked_documents = splitter.split(documents)

Error on document 83
page_content='# Configuring Chainlink Nodes\n\n  * Overview\n  * TOML Configuration\n  * Changes to node configuration starting in v1.1.0 nodes\n  * Essential environment variables\n  * DATABASE_URL\n  * General Node Configuration\n  * CL_CONFIG\n  * CHAIN_TYPE\n  * CHAINLINK_DEV\n  * EXPLORER_ACCESS_KEY\n  * EXPLORER_SECRET\n  * EXPLORER_URL\n  * ROOT\n  * TELEMETRY_INGRESS_UNICONN\n  * TELEMETRY_INGRESS_LOGGING\n  * TELEMETRY_INGRESS_URL\n  * TELEMETRY_INGRESS_SERVER_PUB_KEY\n  * TELEMETRY_INGRESS_BUFFER_SIZE\n  * TELEMETRY_INGRESS_MAX_BATCH_SIZE\n  * TELEMETRY_INGRESS_SEND_INTERVAL\n  * TELEMETRY_INGRESS_SEND_TIMEOUT\n  * TELEMETRY_INGRESS_USE_BATCH_SEND\n  * Chains\n  * SOLANA_ENABLED\n  * EVM_ENABLED\n  * Database Settings\n  * MIGRATE_DATABASE\n  * ORM_MAX_IDLE_CONNS\n  * ORM_MAX_OPEN_CONNS\n  * Database Global Lock\n  * DATABASE_LOCKING_MODE\n  * ADVISORY_LOCK_CHECK_INTERVAL\n  * ADVISORY_LOCK_ID\n  * LEASE_LOCK_DURATION\n  * LEASE_LOCK_REFRESH_INTERVAL\n  *

In [6]:
retriever = TFIDFRetriever.from_documents(chunked_documents, k=2)
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.)
chain = RetrievalQAWithSourcesChain.from_chain_type(retriever=retriever, llm=llm, chain_type="stuff")

In [7]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl", "rb") as f:
    ground_truths = pickle.load(f)

df = pd.DataFrame(ground_truths)

In [10]:
df2 = df.copy()
for i, gt in df2.iterrows():
    try:
        res = chain(gt["question"])
        df2.loc[i, "result"] = res["answer"]
        df2.loc[i, "sources"] = res["sources"]
    except Exception as e:
        print(e)
        print(f"Error on {i}")
        print(gt["question"])

    print(f"Done with {i}")

In [16]:
evaluate_dataframe_answer(df2)

Output()

Button(description='Right', style=ButtonStyle())

Button(description='Wrong', style=ButtonStyle())

Button(description='partial', style=ButtonStyle())

In [18]:
df2.to_csv("/home/marshath/play/chainlink/algovate/test_chainlink/16kmodel.csv", index=False)